# Ray Implementation

In [1]:
import numpy as np
import pandas as pd
import joblib
import ray
from ray.util.joblib import register_ray
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVC

SEED = 42

In [2]:
register_ray()

## Read data

In [3]:
full_data = ray.data.read_csv('/home/ubuntu/data/cleaned_data.csv')

2024-04-14 20:09:38,295	INFO worker.py:1567 -- Connecting to existing Ray cluster at address: 172.31.88.97:6379...
2024-04-14 20:09:38,306	INFO worker.py:1743 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


In [7]:
print(len(full_data.index))

39435236


In [7]:
full_data.head(5)

AttributeError: 'Dataset' object has no attribute 'head'

### Create different sized dataframes to test speed and size scaling

In [4]:
# Ray subset
small_data = full_data.random_sample(fraction=0.1, seed=SEED)

### Split into train and test DO WE NEED SEEDING HERE FOR REPLICATION ACROSS NOTEBOOKS?

In [7]:
# Ray split
# Split data into X and y
X_reg = small_data.select_columns(['dateDelta', 'seatsRemaining', 'totalTravelDistance',
                  'durationSeconds']).to_numpy_refs()

y_reg = small_data.select_columns(['baseFare']).to_numpy_refs()

X_class = small_data.select_columns(['dateDelta', 'seatsRemaining', 'totalTravelDistance',
                  'durationSeconds', 'baseFare']).to_numpy_refs()

y_class = small_data.select_columns(['destinationAirport']).to_numpy_refs()

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=5110)
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=5110)

2024-04-14 20:10:47,774	INFO streaming_executor.py:115 -- Starting execution of Dataset. Full log is in /tmp/ray/session_2024-04-14_20-07-44_864706_1465/logs/ray-data.log
2024-04-14 20:10:47,775	INFO streaming_executor.py:116 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> TaskPoolMapOperator[MapBatches(process_batch)->MapBatches(fn)]



- ReadCSV->SplitBlocks(11) 1:   0%|                                                                           …

- MapBatches(process_batch)->MapBatches(fn) 2:   0%|                                                          …

Running 0:   0%|                                                                                              …

2024-04-14 20:11:27,894	WARNING plan.py:549 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for more details: https://docs.ray.io/en/latest/data/data-internals.html#ray-data-and-tune
2024-04-14 20:11:27,912	INFO streaming_executor.py:115 -- Starting execution of Dataset. Full log is in /tmp/ray/session_2024-04-14_20-07-44_864706_1465/logs/ray-data.log
2024-04-14 20:11:27,915	INFO streaming_executor.py:116 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> TaskPoolMapOperator[MapBatches(process_batch)->MapBatches(fn)]



...

Running 0:   0%|                                                                                              …

2024-04-14 20:12:49,798	INFO streaming_executor.py:115 -- Starting execution of Dataset. Full log is in /tmp/ray/session_2024-04-14_20-07-44_864706_1465/logs/ray-data.log
2024-04-14 20:12:49,799	INFO streaming_executor.py:116 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> TaskPoolMapOperator[MapBatches(process_batch)->MapBatches(fn)]



- ReadCSV->SplitBlocks(11) 1:   0%|                                                                           …

- MapBatches(process_batch)->MapBatches(fn) 2:   0%|                                                          …

Running 0:   0%|                                                                                              …

## KNN Model

### Timing Code

In [18]:
#%%timeit
#KNN model
#ray.init(address="6379")
with joblib.parallel_backend('ray'):
    knn_model = KNeighborsRegressor(n_neighbors=5)
    knn_model.fit(X_train_reg, y_train_reg)
    acc = knn_model.score(X_test_reg.reshape(-1, 1), y_test_reg.reshape(-1, 1))

ValueError: Expected 2D array, got 1D array instead:
array=[ObjectRef(4734df6ace9f4391ffffffffffffffffffffffff0300000001000000)
 ObjectRef(8c05139bcc2b364fffffffffffffffffffffffff0300000001000000)
 ObjectRef(76aa34f5b5bbb7f2ffffffffffffffffffffffff0300000001000000)
 ObjectRef(c8949d20ce9684ffffffffffffffffffffffffff0300000001000000)
 ObjectRef(bd65b3675ddfb966ffffffffffffffffffffffff0300000001000000)
 ObjectRef(5db0db9e5c879fd1ffffffffffffffffffffffff0300000001000000)
 ObjectRef(ea1c0888f0f4f839ffffffffffffffffffffffff0300000001000000)
 ObjectRef(8f766538ab157fbbffffffffffffffffffffffff0300000001000000)
 ObjectRef(01acd62c4eafb4feffffffffffffffffffffffff0300000001000000)
 ObjectRef(b59b34b03f035cacffffffffffffffffffffffff0300000001000000)
 ObjectRef(99bdd76032791cd9ffffffffffffffffffffffff0300000001000000)
 ObjectRef(8ba5a622db55dbd9ffffffffffffffffffffffff0300000001000000)
 ObjectRef(f08ab829d2e27be5ffffffffffffffffffffffff0300000001000000)
 ObjectRef(6a29db808a9d5a64fffffffffffffffff

### Accuracy Code

In [7]:
# Accuracy code
#KNN model
ray.init(address="auto")
with joblib.parallel_backend('ray'):
    knn_model = KNeighborsRegressor(n_neighbors=5)
    knn_model.fit(X_train_reg, y_train_reg)
    acc = knn_model.score(X_test_reg, y_test_reg)

In [8]:
acc

0.6196041454692904

## K-Means Model

### Timing Code

In [7]:
%%timeit
#K-Means
ray.init(address="auto")
with joblib.parallel_backend('ray'):
    kmeans_model = KMeans(n_clusters=2, random_state=5110, n_init="auto")
    kmeans_model.fit(X_train_reg)

8.33 s ± 484 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Accuracy Code for K-MEANS N/A

In [10]:
# Hehe

## Random Forest Model

### Timing Code

In [8]:
%%timeit
#Random Forest
ray.init(address="auto")
with joblib.parallel_backend('ray'):
    rf_model = RandomForestRegressor(max_depth=2, random_state=5110)
    rf_model.fit(X_train_reg, y_train_reg)
   
    y_pred = rf_model.predict(X_test_reg)
    mse = mean_squared_error(y_test_reg, y_pred)

5min 21s ± 2.17 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Accuracy Code

In [12]:
#Random Forest
ray.init(address="auto")
with joblib.parallel_backend('ray'):
    rf_model = RandomForestRegressor(max_depth=2, random_state=5110)
    rf_model.fit(X_train_reg, y_train_reg)
   
    y_pred = rf_model.predict(X_test_reg)
    mse = mean_squared_error(y_test_reg, y_pred)

KeyboardInterrupt: 

In [ ]:
np.sqrt(mse)

In [ ]:
data.baseFare.describe()

In [ ]:
#SVM
ray.init(address="auto")
with joblib.parallel_backend('ray'):
    svm_model = LinearSVC(random_state=5110, multi_class='ovr')
    svm_model.fit(X_train_class, y_train_class)
    acc = svm_model.score(X_test_class, y_test_class)